# Image Offsets: Create offset arrays used by drizzle

The offset tables created by a previous notebook (Offsets_2) are used to generate the X and Y offset arrays used by drizzle.

Each offset table will generate 2 arrays, for X and Y respectively, stored as FITS image extensions of short float type.

The algorithms were developed in the Timing notebook; here, they are cast as callable functions used in a loop to process all images in the sequence.

In [1]:
import os, glob
import numpy as np

import multiprocessing as mp
from multiprocessing import Pool

from astropy.table import Table

import rawpy

In [2]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

## Read last table in sequence, and prototype image

Starting with the last table ensures that we get always the same stars along the entire sequence. Offset tables at the beginning of the sequence may include stars that are dropped later on.

In [3]:
# last table in sequence
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
last = table_list[-1]
offsets_table = Table.read(last)

In [4]:
# prototype image (used to define array size)
image_name = last.split('/')[-1]
image_name = image_name.replace('.offsets_table.fits', '.ARW')
image_name = os.path.join(datadir, image_name)
raw = rawpy.imread(image_name)
imarray = raw.raw_image_visible.astype(float)

# this makes indices consistent with daofind-defined centroids
nx = imarray.shape[1]
ny = imarray.shape[0]

print(imarray.shape)
print(nx, ny)

(2848, 4256)
4256 2848


## Create output arrays and get data from offsets table

In [5]:
offset_array_x = np.asarray(imarray) * 0.0
offset_array_y = np.asarray(imarray) * 0.0

In [6]:
centroid_x = offsets_table['xcentroid'].data
centroid_y = offsets_table['ycentroid'].data
offset_x = offsets_table['xoffset'].data
offset_y = offsets_table['yoffset'].data

## Functions

In [7]:
# Comparison functions
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

# Gets the index of the closest star in table. 
# The differences are in the sense pixel - star centroid.
# The comparison functions define from which quadrant the star is drawn from.
def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0

### Parallelization functions

The offset computation for each individual pixel is prohibitive without parallelization.

In [8]:
# Callback to collect results from workers into sections of the output arrays
def collect_result(results):
    
    x0 = results['x0']
    y0 = results['y0']
    nx = results['size_x']
    ny = results['size_y']

    offset_array_section_x = results['offset_array_x']
    offset_array_section_y = results['offset_array_y']
    
    print("offset_array_x.shape:  ", offset_array_x.shape)
    
    offset_array_x[y0:y0+ny,x0:x0+nx] = offset_array_section_x
    offset_array_y[y0:y0+ny,x0:x0+nx] = offset_array_section_y

class Worker:
    '''
    A class with callable instances that execute the offset calculation
    algorithm over a section of the input image. It provides the callable 
    for the `Pool.apply_async` function, and also holds all parameters 
    necessary to perform the calculation.
    '''
    def __init__(self, x0, y0, size_x, size_y, step_x, step_y, centroid_x, centroid_y, 
                offset_x, offset_y):
        '''
        Parameters:
        
        x0, y0 - top left pixel of the image section designated for this instance
        size_x, size_y - size of the image section
        step_x - step used in the x direction when looping over pixels
        step_y - step used in the y direction when looping over pixels
        centroid_x - 1-D data from the `xcentroid` column in the offsets table 
        centroid_y - 1-D data from the `ycentroid` column in the offsets table 
        offset_x - 1-D data from the `xoffset` column in the offsets table 
        offset_y - 1-D data from the `yoffset` column in the offsets table 
        
        Returns:
        
        dict with output arrays. To be collected by a callback function. 
        '''
        self.x0 = x0
        self.y0 = y0
        self.size_x = size_x
        self.size_y = size_y
        self.step_x = step_x
        self.step_y = step_y

        self.centroid_x = centroid_x
        self.centroid_y = centroid_y
        self.offset_x = offset_x
        self.offset_y = offset_y
        
        # create local output arrays
        self.offset_array_x = np.zeros(shape=(self.size_y, self.size_x))
        self.offset_array_y = np.zeros(shape=(self.size_y, self.size_x))
        
        print(self.offset_array_x.shape)

    def __call__(self):
#         max_x = self.x0 + self.size_x - self.step_x
#         max_y = self.y0 + self.size_y - self.step_y
        
#         print("max: ", max_x, max_y)

#         for i in range(self.x0, max_x, self.step_x):
#             for j in range(self.y0, max_y, self.step_y):

        for i in range(0, self.size_x, self.step_x):
            for j in range(0, self.size_y, self.step_y):
                
#                 print(j)
        # for i in range(2000, 2001):
        #     for j in range(1000, 1003):

                pixel_x = int(i + self.x0)
                pixel_y = int(j + self.y0)

                diff_x = pixel_x - self.centroid_x
                diff_y = pixel_y - self.centroid_y

                index = np.array(range(4), dtype=int)
                dist  = np.array(range(4), dtype=float)

                # get index and distance of the closest star, one per quadrant
                index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
                index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
                index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
                index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)

                # need a cleanup here. Negative indices, zeroed distances.

                # weighted average of the offset values. The weight is the inverse distance pixel-star.
                sumweights = 0.0
                for k in range(len(dist)):
                    if dist[k] > 0.:
                        sumweights += 1./dist[k]
                        
                weighted_offset_x = 0.0
                weighted_offset_y = 0.0

                for k in range(len(index)):
                    if index[k] > 0:
                        weighted_offset_x += self.offset_x[index[k]] * (1./dist[k] / sumweights)
                        weighted_offset_y += self.offset_y[index[k]] * (1./dist[k] / sumweights)

                self.offset_array_x[j][i] = weighted_offset_x
                self.offset_array_y[j][i] = weighted_offset_y
        
        # return the local output arrays with offsets for this section of the image,
        # plus metadata to locate the section on the full offsets arrays.
        return {'x0': self.x0,
                'y0': self.y0,
                'size_x': self.size_x,
                'size_y': self.size_y,
                'offset_array_x': self.offset_array_x,
                'offset_array_y': self.offset_array_y
               }

## Populate output arays

In [9]:
# each thread runs over a subsection of the image
# w1 = Worker(0,           0, int(nx/4), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y)
# w2 = Worker(int(nx/4),   0, int(nx/4), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y)
# w3 = Worker(int(nx/2),   0, int(nx/4), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y)
# w4 = Worker(int(3*nx/4), 0, int(nx/4), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y)

# number of processors
nproc = 8

results = []
pool = Pool(nproc)

for p in range(nproc):
    worker = Worker(int(p*nx/nproc), 0, int(nx/nproc), ny, 2, 2, centroid_x, centroid_y, offset_x, offset_y)
    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)
    
    
# r = pool.apply_async(w1, callback=collect_result)
# results.append(r)
# r = pool.apply_async(w2, callback=collect_result)
# results.append(r)
# r = pool.apply_async(w3, callback=collect_result)
# results.append(r)
# r = pool.apply_async(w4, callback=collect_result)
# results.append(r)

for r in results:
    r.wait()

pool.close()

# alternate, non-threaded version to test the worker code
# collect_result(w1())
# collect_result(w2())

(2848, 532)
(2848, 532)
(2848, 532)
(2848, 532)
(2848, 532)
(2848, 532)
(2848, 532)
(2848, 532)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)
offset_array_x.shape:   (2848, 4256)


In [10]:
offset_array_x[1500:1510,1500:1510]

array([[-24.72381179,   0.        , -24.72757489,   0.        ,
        -24.73144622,   0.        , -24.8062214 ,   0.        ,
        -24.80472781,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-24.69539601,   0.        , -24.69874399,   0.        ,
        -24.70226868,   0.        , -24.78021813,   0.        ,
        -24.77850063,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-24.66661972,   0.        , -24.6696302 ,   0.        ,
        -24.67289365,   0.        , -24.57851558,   0.        ,
        -24.57737961,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-2

In [11]:
offset_array_x[1500:1510,3500:3510]

array([[-33.84440213,   0.        , -33.87840953,   0.        ,
        -33.91298056,   0.        , -33.94809179,   0.        ,
        -33.98371768,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-33.82014   ,   0.        , -33.85398904,   0.        ,
        -33.88839183,   0.        , -33.92332209,   0.        ,
        -33.95875131,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-33.79520025,   0.        , -33.82890549,   0.        ,
        -33.8631557 ,   0.        , -33.89792172,   0.        ,
        -33.93317221,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-3

In [12]:
# offset_array_x_2[1500:1510,1500:1510]

In [13]:
# offset_array_x_2[1500:1510,3500:3510]